In [1]:
from os import path
from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
BASE_PATH = "/home/divyanshu/Desktop/Kaggle/Facial_Expressions/fer13"
INPUT_PATH = path.sep.join([BASE_PATH,"fer2013/fer2013.csv"])
print(INPUT_PATH)

Using TensorFlow backend.


/home/divyanshu/Desktop/Kaggle/Facial_Expressions/fer13/fer2013/fer2013.csv


In [2]:
n_classes = 7
TRAIN_HDF5 = path.sep.join([BASE_PATH, "hdf5/train.hdf5"])
print(TRAIN_HDF5)
VAL_HDF5 = path.sep.join([BASE_PATH, "hdf5/val.hdf5"])
TEST_HDF5 = path.sep.join([BASE_PATH, "hdf5/test.hdf5"])

/home/divyanshu/Desktop/Kaggle/Facial_Expressions/fer13/hdf5/train.hdf5


In [3]:
BATCH_SIZE = 128
OUTPUT_PATH = path.sep.join([BASE_PATH, "output"])

In [4]:
from hdf5datasetwriter import HDF5DatasetWriter
import numpy as np

In [5]:
f = open(INPUT_PATH)
f.__next__()
(trainImages, trainLabels) = ([], [])
(valImages, valLabels) = ([], [])
(testImages, testLabels) = ([], [])
labels = []
imlist = []
for row in f:
    (label, image, usage) = row.strip().split(",")
    label = int(label)
    image = np.array(image.split(" "), dtype = "uint8")
    image = image.reshape((1, 48, 48, 1))
    if usage == "Training":
        trainImages.append(image)
        trainLabels.append(label)
    elif usage == "PrivateTest":
        valImages.append(image)
        valLabels.append(label)
    else:
        testImages.append(image)
        testLabels.append(label)
datasets = [
(trainImages, trainLabels, TRAIN_HDF5),
(valImages, valLabels, VAL_HDF5),
(testImages, testLabels, TEST_HDF5)]

In [6]:
np.shape(trainImages)

(28709, 1, 48, 48, 1)

In [7]:
for (images, labels, outputPath) in datasets:
    writer = HDF5DatasetWriter((len(images), 48, 48, 1), outputPath)
    for (image, label) in zip(images, labels):
        writer.add([image], [label])
    writer.close()
f.close()

ValueError: Path already exists. Manually delete it

In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding="same", kernel_initializer="he_normal", input_shape=(48,48,1)))
#model.add(ELU())
#model.add(BatchNormalization(axis=-1))
#model.add(Conv2D(32, (3, 3), kernel_initializer="he_normal",
#padding="same"))
#model.add(ELU())
#model.add(BatchNormalization(axis=-1))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
#model.add(Conv2D(64, (3, 3), kernel_initializer="he_normal",
#padding="same"))
#model.add(ELU())
#model.add(BatchNormalization(axis=-1))
#model.add(Conv2D(64, (3, 3), kernel_initializer="he_normal",
#padding="same"))
#model.add(ELU())
#model.add(BatchNormalization(axis=-1))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
#model.add(Conv2D(128, (3, 3), kernel_initializer="he_normal",
#padding="same"))
#model.add(ELU())
#model.add(BatchNormalization(axis=-1))
#model.add(Conv2D(128, (3, 3), kernel_initializer="he_normal",
#padding="same"))
#model.add(ELU())
#model.add(BatchNormalization(axis=-1))
#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
#model.add(Flatten())
#model.add(Dense(64, kernel_initializer="he_normal"))
#model.add(ELU())
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
#model.add(Dense(64, kernel_initializer="he_normal"))
#model.add(ELU())
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
#model.add(Dense(n_classes, kernel_initializer="he_normal", activation="softmax"))
opt = Adam(lr=1e-3)
model.compile(loss="categorical_crossentropy", optimizer=opt,
metrics=["accuracy"])

In [ ]:
from imagetoarray_preprocessor import ImageToArrayPreprocessor
from trainingmonitor import TrainingMonitor
from hdf5datasetgenerator import HDF5DatasetGenerator
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.models import load_model
import keras.backend as K
import argparse
import os

In [ ]:
trainAug = ImageDataGenerator(rotation_range=10, zoom_range=0.1,
horizontal_flip=True, rescale=1 / 255.0, fill_mode="nearest")
valAug = ImageDataGenerator(rescale=1 / 255.0)
iap = ImageToArrayPreprocessor()
trainGen = HDF5DatasetGenerator(TRAIN_HDF5, BATCH_SIZE,
aug=trainAug, preprocessors=[iap], classes=n_classes)
valGen = HDF5DatasetGenerator(VAL_HDF5, BATCH_SIZE,
aug=valAug, preprocessors=[iap], classes=n_classes)

In [ ]:
from keras.callbacks import Callback
import os

class EpochCheckpoint(Callback):
    def __init__(self, outputPath, every=5, startAt=0):
        # call the parent constructor
        super(Callback, self).__init__()

        # store the base output path for the model, the number of
        # epochs that must pass before the model is serialized to
        # disk and the current epoch value
        self.outputPath = outputPath
        self.every = every
        self.intEpoch = startAt

    def on_epoch_end(self, epoch, logs={}):
        # check to see if the model should be serialized to disk
        if (self.intEpoch + 1) % self.every == 0:
            p = os.path.sep.join([self.outputPath,"epoch_{}.hdf5".format(self.intEpoch + 1)])
            self.model.save(p, overwrite=True)

        # increment the internal epoch counter
        self.intEpoch += 1

In [ ]:
model.summary()

In [ ]:
figPath = os.path.sep.join([OUTPUT_PATH,
"vggnet_emotion.png"])
jsonPath = os.path.sep.join([OUTPUT_PATH,
"vggnet_emotion.json"])
#callbacks = [
# EpochCheckpoint("/home/divyanshu/Desktop/Kaggle/Facial_Expressions/checkpoints", every=5,startAt=0),
# TrainingMonitor(figPath, jsonPath, 0)]

In [ ]:
trainImages[0]

In [ ]:
print(np.shape(np.asarray(trainImages[0])))

In [ ]:
x = np.asarray(trainImages[0])
y = np.asarray(trainLabels[0])
x.ndim

In [ ]:
model.fit(x,y)